https://qiita.com/akita0724/items/694cc4e142d8d1d7fdaa

# DL部

| 試験区分                         | コード |
|-----------------------------------|------|
| 応用情報技術者試験                 | ap    |
| ITストラテジスト試験               | st    |
| システムアーキテクト試験             | sa    |
| プロジェクトマネージャ試験           | pw    |
| ネットワークスペシャリスト試験       | nw    |
| データベーススペシャリスト試験         | db    |
| エンベデッドシステムスペシャリスト試験 | es    |
| ITサービスマネージャ試験           | sm    |
| システム監査技術者試験             | au    |
| 情報処理安全確保支援士試験           | sc    |

In [2]:
import requests
import shutil
import tempfile
import os
from os import makedirs, path
from time import sleep
from urllib.request import urlretrieve



# 過去問のURLを生成するための情報
base_url = "https://www.ipa.go.jp/shiken/mondai-kaiotu/"
ids = [
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06a_"},
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06h_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05a_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05h_"},
    {"id": "gmcbt80000008smf-att", "filename_ids": "2022r04a_"},
    {"id": "gmcbt80000009sgk-att", "filename_ids": "2022r04h_"},
    {"id": "gmcbt8000000apad-att", "filename_ids": "2021r03a_"},
    {"id": "gmcbt8000000d5ru-att", "filename_ids": "2021r03h_"},
    {"id": "gmcbt8000000d05l-att", "filename_ids": "2020r02o_"},
    {"id": "gmcbt8000000dict-att", "filename_ids": "2019r01a_"},
    {"id": "gmcbt8000000ddiw-att", "filename_ids": "2019h31h_"},
    {"id": "gmcbt8000000f01f-att", "filename_ids": "2018h30a_"},
    {"id": "gmcbt8000000fabr-att", "filename_ids": "2018h30h_"},
    {"id": "gmcbt8000000fqpm-att", "filename_ids": "2017h29a_"},
    {"id": "gmcbt8000000fzx1-att", "filename_ids": "2017h29h_"},
    {"id": "gmcbt8000000g6fw-att", "filename_ids": "2016h28a_"},
    {"id": "gmcbt8000000gn5o-att", "filename_ids": "2016h28h_"},
    {"id": "gmcbt8000000gxj0-att", "filename_ids": "2015h27a_"},
    {"id": "ug65p90000000f52-att", "filename_ids": "2015h27h_"},
    {"id": "ug65p90000000ye5-att", "filename_ids": "2014h26a_"},
    {"id": "ug65p90000001dzu-att", "filename_ids": "2014h26h_"},
    {"id": "ug65p900000027za-att", "filename_ids": "2013h25a_"},
    {"id": "ug65p90000002e6g-att", "filename_ids": "2013h25h_"},
    {"id": "ug65p90000002h5m-att", "filename_ids": "2012h24a_"},
    {"id": "ug65p900000038er-att", "filename_ids": "2012h24h_"},
    {"id": "ug65p90000003ojp-att", "filename_ids": "2011h23a_"},
    {"id": "ug65p90000003ya2-att", "filename_ids": "2011h23tokubetsu_"},
    {"id": "ug65p90000004d6f-att", "filename_ids": "2010h22a_"},
    {"id": "ug65p90000004n2z-att", "filename_ids": "2010h22h_"},
    {"id": "gmcbt8000000f3yi-att", "filename_ids": "2009h21a_"},
    {"id": "ug65p90000009bhl-att", "filename_ids": "2009h21h_"},
]

# ファイルの種類とファイル名のパターン
file_types = {
    "am": ["am_qs", "am_ans"],
    "pm": ["pm_qs", "pm_ans", "pm_cmnt"],
    "am1": ["am1_qs", "am1_ans"],
    "am2": ["am2_qs", "am2_ans"],
    "pm1": ["pm1_qs", "pm1_ans", "pm1_cmnt"],
    "pm2": ["pm2_qs", "pm2_ans", "pm2_cmnt"],
}

# ファイル名のパターン
filename_codes = {
    "qs": "問題",
    "ans": "解答",
    "cmnt": "採点講評",
}

# ファイルをダウンロードする関数
def dl_file(url, filename, kamoku_code, year, season_name):
    # ダウンロード前にファイルパスを生成
    output_dir = f"./DL/{kamoku_code.upper()}/{year}/{season_name}"
    output_path = path.join(output_dir, filename)

    if path.exists(output_path):
        print(f"{filename} は既に存在するため、ダウンロードをスキップします。")
        return True

    response = requests.head(url)
    if response.status_code == 200:
        # ダウンロード前にフォルダを作成
        if not path.exists(output_dir):
            makedirs(output_dir)
        urlretrieve(url, output_path)
        print(f"{filename} を保存しました。")
        sleep(3)
        return True
    else:
        print(f"{filename} は見つかりませんでした。")
        return False

# フォルダを作成する関数
def create_output_dir(kamoku_code, year, season_name):
    kamoku_name_for_folder = kamoku_code.upper()
    season_name_for_folder = {"春季": "Spring", "秋季": "Autumn", "特別": "Special"}.get(season_name, "Special")
    output_dir = f"./DL/{kamoku_name_for_folder}/{year}/{season_name_for_folder}"
    return output_dir


def dl_mondai(kamoku_code):
    kamoku_name = kamoku_codes[kamoku_code]

    for data in ids:
        year = data["filename_ids"][:4]
        season = data["filename_ids"][7]
        season_name = {"h": "春季", "a": "秋季", "o": "特別"}.get(season, "特別")

        # 午後の問題ファイルが存在するか確認するためのフラグ
        pm_exists = False

        for time_part in ["pm", "am"]: # 午後を先に処理するように変更
            # 午前I/II、午後I/IIのファイルがあるか確認
            for i in range(1, 3):
                new_time_part = f"{time_part}{i}"

                # 午前問題の場合、"ap" または "koudo" を使用
                am_code = "koudo" if kamoku_code != "ap" and time_part == "am" and i == 1 else kamoku_code

                # URLとファイル名を生成 (問題)
                new_url_qs = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_qs.pdf"
                # ファイル名に試験名を追加
                new_filename_qs = f"{kamoku_name}_{year}{season_name}_{new_time_part.upper()}_{filename_codes['qs']}.pdf"

                # URLとファイル名を生成 (解答)
                new_url_ans = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_ans.pdf"
                # ファイル名に試験名を追加
                new_filename_ans = f"{kamoku_name}_{year}{season_name}_{new_time_part.upper()}_{filename_codes['ans']}.pdf"

                # 午後の場合、問題ファイルが存在するか確認
                if time_part == "pm":
                    output_path_qs = new_filename_qs
                    if i == 1: # PM1 の場合のみ存在確認
                        if not path.exists(output_path_qs): # ファイルが存在しない場合のみダウンロードを試みる
                            if dl_file(new_url_qs, output_path_qs, kamoku_code, year, season_name):
                                pm_exists = True # 午後の問題が存在する場合、フラグを立てる
                                # 解答をダウンロード
                                output_path_ans = new_filename_ans
                                dl_file(new_url_ans, output_path_ans, kamoku_code, year, season_name)

                                # 採点講評のURLとファイル名を生成 (午後I/IIのみ)
                                if time_part == "pm":
                                    new_url_cmnt = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_cmnt.pdf"
                                    # ファイル名に試験名を追加
                                    new_filename_cmnt = f"{kamoku_name}_{year}{season_name}_{new_time_part.upper()}_{filename_codes['cmnt']}.pdf"
                                    output_path_cmnt = new_filename_cmnt
                                    dl_file(new_url_cmnt, output_path_cmnt, kamoku_code, year, season_name)
                            else:
                                # PM1 が存在しない場合、PM のみ存在するか確認
                                new_time_part_pm = "pm"
                                new_url_qs_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_qs.pdf"
                                # ファイル名に試験名を追加
                                new_filename_qs_pm = f"{kamoku_name}_{year}{season_name}_{new_time_part_pm.upper()}_{filename_codes['qs']}.pdf"
                                output_path_qs_pm = new_filename_qs_pm
                                if not path.exists(output_path_qs_pm):
                                    if dl_file(new_url_qs_pm, output_path_qs_pm, kamoku_code, year, season_name):
                                        pm_exists = True
                                        new_url_ans_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_ans.pdf"
                                        # ファイル名に試験名を追加
                                        new_filename_ans_pm = f"{kamoku_name}_{year}{season_name}_{new_time_part_pm.upper()}_{filename_codes['ans']}.pdf"
                                        output_path_ans_pm = new_filename_ans_pm
                                        dl_file(new_url_ans_pm, output_path_ans_pm, kamoku_code, year, season_name)
                                        new_url_cmnt_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_cmnt.pdf"
                                        # ファイル名に試験名を追加
                                        new_filename_cmnt_pm = f"{kamoku_name}_{year}{season_name}_{new_time_part_pm.upper()}_{filename_codes['cmnt']}.pdf"
                                        output_path_cmnt_pm = new_filename_cmnt_pm
                                        dl_file(new_url_cmnt_pm, output_path_cmnt_pm, kamoku_code, year, season_name)
                                    else:
                                        print(f"{year}{season_name}の午後問題が存在しないため、午前のダウンロードをスキップします。")
                                        break # 午後が存在しない場合、午前のダウンロードをスキップ
                        else:
                            pm_exists = True # PM1 が存在する場合はフラグを立てる
                    elif pm_exists: # PM2 の場合、PM1 が存在する場合のみダウンロード
                        # PM2 の問題、解答、採点講評をダウンロード
                        output_path_qs = new_filename_qs
                        dl_file(new_url_qs, output_path_qs, kamoku_code, year, season_name)
                        output_path_ans = new_filename_ans
                        dl_file(new_url_ans, output_path_ans, kamoku_code, year, season_name)
                        if time_part == "pm":
                            new_url_cmnt = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_cmnt.pdf"
                            # ファイル名に試験名を追加
                            new_filename_cmnt = f"{kamoku_name}_{year}{season_name}_{new_time_part.upper()}_{filename_codes['cmnt']}.pdf"
                            output_path_cmnt = new_filename_cmnt
                            dl_file(new_url_cmnt, output_path_cmnt, kamoku_code, year, season_name)
                # 午前の場合、ダウンロードを実行
                elif time_part == "am" and pm_exists: # 午後が存在する場合のみ、午前をダウンロード
                    output_path_qs = new_filename_qs
                    if dl_file(new_url_qs, output_path_qs, kamoku_code, year, season_name):
                        # 解答をダウンロード
                        output_path_ans = new_filename_ans
                        dl_file(new_url_ans, output_path_ans, kamoku_code, year, season_name)

            # am/pm のファイルがないか確認
            if pm_exists: # 午後が存在する場合のみ、am/pmのファイルを確認
                for file_type_suffix in file_types[time_part]:
                    # 午前問題の場合、"ap" または "koudo" を使用
                    am_code = "koudo" if kamoku_code != "ap" and time_part == "am" else kamoku_code

                    # 過去問のURLを生成
                    url = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{file_type_suffix}.pdf"

                    # ファイル名のパターンを決定
                    if file_type_suffix.endswith("_cmnt"):
                        filename_pattern_key = "cmnt"  # 採点講評の場合
                    else:
                        filename_pattern_key = file_type_suffix.split("_")[1]

                    # 保存するファイル名を生成
                    # ファイル名に試験名を追加
                    filename = f"{kamoku_name}_{year}{season_name}-{time_part.upper()}_{filename_codes[filename_pattern_key]}.pdf"
                    output_path = filename
                    dl_file(url, output_path, kamoku_code, year, season_name)

        if not pm_exists:
            print(f"{year}{season_name}の試験は存在しないため、フォルダを作成しません。")
            continue # 午後が存在しない場合、フォルダを作成しない

    print(f"{kamoku_name}のダウンロードが完了しました。")

# プログラムのエントリポイント
if __name__ == "__main__":
    while True:
        inp = input(
            "ダウンロードしたい試験区分のコードを小文字で入力してください（例：ap）：\n修了する場合はqを入力してください。"
        )
        if inp in kamoku_codes:
            dl_mondai(inp)
            break
        elif inp == "q":
            print("終了します。")
            exit()
        else:
            print("試験区分のコードを入力してください。")

ITストラテジスト試験_2024秋季_PM1_問題.pdf は見つかりませんでした。
ITストラテジスト試験_2024秋季_PM_問題.pdf は見つかりませんでした。
2024秋季の午後問題が存在しないため、午前のダウンロードをスキップします。
2024秋季の試験は存在しないため、フォルダを作成しません。
ITストラテジスト試験_2024春季_PM1_問題.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_PM1_解答.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_PM1_採点講評.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_PM2_問題.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_PM2_解答.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_PM2_採点講評.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季-PM_問題.pdf は見つかりませんでした。
ITストラテジスト試験_2024春季-PM_解答.pdf は見つかりませんでした。
ITストラテジスト試験_2024春季-PM_採点講評.pdf は見つかりませんでした。
ITストラテジスト試験_2024春季_AM1_問題.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_AM1_解答.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_AM2_問題.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季_AM2_解答.pdf は既に存在するため、ダウンロードをスキップします。
ITストラテジスト試験_2024春季-AM_問題.pdf は見つかりませんでした。
ITストラテジスト試験_2024春季-AM_解答.pdf は見つかりませんでした。
ITストラテジスト試験_2023秋季_PM1_問題.pdf は見つかりませんでした。
ITストラテジスト試験_2023秋季_PM_問題.p

読み取り
https://qiita.com/kccs_hiroshi-tatsuwaki/items/f3277a157acf965c33e8

# PDF→画像部


## pdf2imageで分割

In [3]:
import os
import shutil
import tempfile
from os import makedirs, path
from time import sleep
from urllib.request import urlretrieve
import subprocess
from pdf2image import convert_from_path

# ファイル名のパターン
filename_codes = {
    "qs": "問題",
    "ans": "解答",
    "cmnt": "採点講評",
}

# PDFを画像に変換する関数
def convert_pdf_to_image(pdf_path, output_dir_path):
    os.makedirs(output_dir_path, exist_ok=True)

    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        output_path = f"{output_dir_path}/page_{i + 1}.png"
        image.save(output_path, "PNG")
        print(f"Saved: {output_path}")

# ダウンロードしたPDFを画像に変換する処理
def process_pdfs():
    dl_dir = "./DL"
    png_dir = "./PNG"
    os.makedirs(png_dir, exist_ok=True)

    for root, dirs, files in os.walk(dl_dir):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                
                # ファイル名から拡張子を除いた部分を取得
                file_name_without_ext = os.path.splitext(file)[0]
                
                # ファイル名からフォルダ名を生成する関数
                def generate_folder_name(file_name):
                    parts = file_name.split('_')
                    if len(parts) < 3:
                        return file_name
                    
                    kamoku_name = parts[0]
                    year_season = parts[1]
                    time_part = parts[2]
                    
                    kamoku_code = ""
                    for code, name in kamoku_codes.items():
                        if name == kamoku_name:
                            kamoku_code = code.upper()
                            break
                    
                    year = year_season[:4]
                    season = year_season[4:6]
                    season_code = {"春季": "S", "秋季": "A", "特別": "X"}.get(season, "X")
                    
                    file_type_part = parts[3] if len(parts) > 3 else ""
                    
                    file_type_code = ""
                    for key, value in filename_codes.items():
                        if value in file_name:
                            file_type_code = key
                            break
                    
                    if not file_type_code:
                        # ファイル名から直接抽出を試みる
                        for key, value in filename_codes.items():
                            if file_name.endswith(f"_{value}"):
                                file_type_code = key
                                break
                        else:
                            file_type_code = "unknown"
                    
                    # time_part から AM/PM を抽出
                    time_part_code = time_part.upper()
                    
                    return f"{kamoku_code}/{year}/{season_code}/{time_part_code}/{file_type_code}"
                
                # ファイル名からフォルダ名を生成
                output_dir_name = generate_folder_name(file_name_without_ext)
                output_dir = os.path.join(png_dir, output_dir_name)
                
                # フォルダが存在するか確認
                if os.path.exists(output_dir):
                    print(f"{output_dir} は既に存在するため、変換をスキップします。")
                    continue
                
                os.makedirs(output_dir, exist_ok=True)
                convert_pdf_to_image(pdf_path, output_dir)
                print(f"{pdf_path} の変換が完了しました。")
    print("全てのPDFファイルの変換が完了しました。")

if __name__ == "__main__":
    process_pdfs()

./PNG\ST/2009/A/AM1/qs は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/AM1/ans は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/AM2/qs は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/AM2/ans は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM1/qs は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM1/cmnt は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM1/ans は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM2/qs は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM2/cmnt は既に存在するため、変換をスキップします。
./PNG\ST/2009/A/PM2/ans は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/AM1/qs は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/AM1/ans は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/AM2/qs は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/AM2/ans は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM1/qs は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM1/cmnt は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM1/ans は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM2/qs は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM2/cmnt は既に存在するため、変換をスキップします。
./PNG\ST/2010/A/PM2/ans は既に存在するため、変換をスキップします。
./PNG\ST/2011/A/AM1/qs は既に存在するため、変換をスキップします。
./PNG\ST/2011/A/AM1/ans は既に存在するため、変換をスキ

# yomitoku部

## yomitoku[GPU]環境整備部
ローカル動作の為に環境整備しているため、動作環境に合わせて変更の必要あり。  
yomitoku[GPU]動作環境：pytorch は 2.5 以上のバージョンに対応しています。その関係で CUDA11.8 以上

今回はpytorch2.5,cuda12.4（を導入済みPCにて指定先だけ変更）

In [1]:
!pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
!set CUDA_PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.4
!set PATH=%CUDA_PATH%\bin;%CUDA_PATH%\libnvvp;%PATH%
!set CUDA_VISIBLE_DEVICES=0 # 複数のGPUがある場合には使うGPUを選択

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

!nvidia-smi

True
12.4
Wed Jan  8 08:27:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080      WDDM  |   00000000:2B:00.0  On |                  N/A |
|  0%   52C    P0             55W /  320W |    1960MiB /  16376MiB |      6%      Default |
|                                         |                        |                  N/A |
+-------------------------------------

In [3]:
!pip install yomitoku[gpu]

   ---------------------------------------- 0.0/279.7 MB ? eta -:--:--
    --------------------------------------- 4.7/279.7 MB 28.6 MB/s eta 0:00:10
   - -------------------------------------- 11.3/279.7 MB 29.4 MB/s eta 0:00:10
   -- ------------------------------------- 17.6/279.7 MB 29.9 MB/s eta 0:00:09
   --- ------------------------------------ 23.6/279.7 MB 29.9 MB/s eta 0:00:09
   --- ------------------------------------ 27.3/279.7 MB 29.8 MB/s eta 0:00:09
   ---- ----------------------------------- 31.2/279.7 MB 26.1 MB/s eta 0:00:10
   ---- ----------------------------------- 34.6/279.7 MB 25.6 MB/s eta 0:00:10
   ----- ---------------------------------- 35.4/279.7 MB 21.6 MB/s eta 0:00:12
   ----- ---------------------------------- 35.9/279.7 MB 19.7 MB/s eta 0:00:13
   ----- ---------------------------------- 40.6/279.7 MB 19.9 MB/s eta 0:00:13
   ------ --------------------------------- 44.0/279.7 MB 19.6 MB/s eta 0:00:13
   ------ --------------------------------- 46.7/2

## yomitoku実行部


基本コマンド
!yomitoku PNG/ST -f md - TXT -v --figure 

yomitokuコマンドに与えるファイル指定は、日本語不可。出力先は可

${path_data} 解析対象の画像が含まれたディレクトリか画像ファイルのパスを直接して指定してください。ディレクトリを対象とした場合はディレクトリのサブディレクトリ内の画像も含めて処理を実行します。
-f, --format 出力形式のファイルフォーマットを指定します。(json, csv, html, md をサポート)
-o, --outdir 出力先のディレクトリ名を指定します。存在しない場合は新規で作成されます。
-v, --vis を指定すると解析結果を可視化した画像を出力します。
-l, --lite を指定すると軽量モデルで推論を実行します。通常より高速に推論できますが、若干、精度が低下する可能性があります。
-d, --device モデルを実行するためのデバイスを指定します。gpu が利用できない場合は cpu で推論が実行されます。(デフォルト: cuda)
--ignore_line_break 画像の改行位置を無視して、段落内の文章を連結して返します。（デフォルト：画像通りの改行位置位置で改行します。）
--figure_letter 検出した図表に含まれる文字も出力ファイルにエクスポートします。
--figure 検出した図、画像を出力ファイルにエクスポートします。
--encoding エクスポートする出力ファイルの文字エンコーディングを指定します。サポートされていない文字コードが含まれる場合は、その文字を無視します。(utf-8, utf-8-sig, shift-jis, enc-jp, cp932)

In [10]:
import os
import shutil
import tempfile
from os import makedirs, path
from time import sleep
from urllib.request import urlretrieve
import subprocess
from pdf2image import convert_from_path

def run_yomitoku(target_kamoku, output_format="md", visualize=False, use_lite_model=False, figure=True):
    png_dir = "./PNG"
    output_base_dir = "./TXT"

    target_kamoku_upper = target_kamoku.upper()
    for root, dirs, files in os.walk(png_dir):
        # ディレクトリ名にターゲットの試験区分が含まれているか確認
        if target_kamoku_upper not in root.upper():
            continue
        for file in files:
            if file.lower().endswith(".png"):
                # ファイルのフルパスを取得
                file_path = os.path.join(root, file)
                
                # 出力先のディレクトリを生成
                relative_path = os.path.relpath(root, png_dir)
                output_dir = os.path.join(output_base_dir, relative_path)
                os.makedirs(output_dir, exist_ok=True)
                
                # yomitokuのコマンドを生成
                command = ["yomitoku", file_path, "-f", output_format, "-o", output_dir]
                if visualize:
                    command.append("-v")
                if use_lite_model:
                    command.append("--lite")
                if figure:
                    command.append("--figure")
                
                # コマンドを実行
                try:
                    result = subprocess.run(command, check=True, capture_output=True, text=True)
                    print(f"Processed: {file_path} -> {output_dir}")
                    print(result.stdout)
                    print(result.stderr)
                except subprocess.CalledProcessError as e:
                    print(f"Error processing {file_path}: {e}")
                    print(e.stderr)

if __name__ == "__main__":
    while True:
        target_kamoku = input("テキスト変換したい試験区分のコードを小文字で入力してください（例：ap）：\n終了する場合はqを入力してください。")
        if target_kamoku: # 入力された文字列が空でないか確認
            run_yomitoku(target_kamoku)
            break
        elif target_kamoku == "q":
            print("終了します。")
            exit()
        else:
            print("試験区分のコードを入力してください。")

Processed: ./PNG\ST\2009\A\AM1\ans\page_1.png -> ./TXT\ST\2009\A\AM1\ans

2025-01-08 19:50:18,000 - yomitoku.base - INFO - Initialize TextDetector
2025-01-08 19:50:18,885 - yomitoku.base - INFO - Initialize TextRecognizer
2025-01-08 19:50:20,415 - yomitoku.base - INFO - Initialize LayoutParser
2025-01-08 19:50:21,147 - yomitoku.base - INFO - Initialize TableStructureRecognizer
2025-01-08 19:50:22,438 - yomitoku.cli.main - INFO - Output directory: ./TXT\ST\2009\A\AM1\ans
2025-01-08 19:50:22,438 - yomitoku.cli.main - INFO - Processing file: PNG\ST\2009\A\AM1\ans\page_1.png
2025-01-08 19:50:22,889 - yomitoku.base - INFO - TextDetector __call__ elapsed_time: 0.4181935787200928
2025-01-08 19:50:22,995 - yomitoku.base - INFO - LayoutParser __call__ elapsed_time: 0.5237143039703369
2025-01-08 19:50:23,192 - yomitoku.base - INFO - TableStructureRecognizer __call__ elapsed_time: 0.19703292846679688
2025-01-08 19:50:24,218 - yomitoku.base - INFO - TextRecognizer __call__ elapsed_time: 1.01885581